In [ ]:
import pandas as pd
kr = pd.read_csv("/data/uijih/detection/8b-inst-full-kr.csv")
en = pd.read_csv("/data/uijih/detection/8b-inst-full-en.csv")

In [ ]:
kr = pd.DataFrame(kr)
en = pd.DataFrame(en)

#count_k = kr['ans'].value_counts().get('o', 0)
count_e = en['ans'].value_counts().get('o', 0)
print(count_e)

In [ ]:
print(count_k, count_e)

In [ ]:
count_k = kr['ans'].value_counts().get('x', 0)
count_e = en['ans'].value_counts().get('x', 0)
print(count_k, count_e)

In [ ]:
count_k = kr['ans'].value_counts().get('m', 0)
count_e = en['ans'].value_counts().get('m', 0)
print(count_k, count_e)

In [ ]:
pip install thefuzz[speedup]

In [ ]:
import pandas as pd
from difflib import SequenceMatcher 

dataset = pd.read_csv('/data/uijih/previous/Seed50_for_Parallel_Dataset_ENKR_idiomKB_0.8_example.csv')
results = pd.read_csv('/data/uijih/detection/70b-inst-kr-2.csv')

# 0~1이고 높을수록 유사
SIMILARITY_THRESHOLD = 0.6
matched_results = []

for prediction in results['Model_Prediction']:
    print(prediction)
    print("="*20)
    highest_similarity = 0
    best_match = None

    # 전처리
    #cleaned_prediction = prediction.replace('<|end_header_id>', '').strip()  
    # kr
    prediction = str(prediction).strip()  
    for idiom in dataset['KR_Idiom']:
        #prediction_lower = cleaned_prediction.lower()
        #idiom_lower = idiom.lower()        
        # 유사도 계산
        #similarity = SequenceMatcher(None, prediction_lower, idiom_lower).ratio()
        # kr
        idiom = str(idiom).strip()  
        print(idiom, prediction)
        similarity = SequenceMatcher(None, str(prediction), idiom).ratio()
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = idiom

    if highest_similarity >= SIMILARITY_THRESHOLD:
        matched_results.append({
            #'Model_Prediction': cleaned_prediction,
            'Model_Prediction': prediction,
            'Matched_Idiom': best_match,
            'Similarity_Score': highest_similarity
        })
    else:
        matched_results.append({
            #'Model_Prediction': cleaned_prediction,
            'Model_Prediction': prediction,
            'Matched_Idiom': None,
            'Similarity_Score': None
        })

matched_df = pd.DataFrame(matched_results)
matched_df.to_csv('matched_results.csv', index=False)

print(matched_df)

In [ ]:
sum(matched_df['Matched_Idiom']==None)

# 8b 2 inference한거

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import random

# 1. 추론 함수 정의
def inference(model_path, tokenizer, input_data, system_prompts_en, labels=None, language='en'):
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
    model.eval()

    # 영어 프롬프트 템플릿
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

{user_input}<|eot_id|>
<|start_header_id|>assistant<|end_header_id>
"""

    results = []
    for idx, sentence in enumerate(input_data):
        # 영어 또는 한국어 프롬프트 사용
        system_prompt = random.choice(system_prompts_en)
        
        if language == 'en':
            user_input = f'Is there an idiom in the sentence "{sentence}"? If yes, return only the detected idiom. Generate just the idiom in its original form without any additional explanation or text. If there is no idiom, answer "none".'
        # else:  # 한국어 문장에 대해 처리
        #     user_input = f'"{sentence}"라는 문장에서 관용구가 있니? 있으면 관용구만 알려줘. 없으면 "없음"이라고 대답해.'

        # 프롬프트 템플릿에 시스템 프롬프트와 사용자 입력 삽입
        input_prompt = prompt_template.format(
            system_prompt=system_prompt,
            user_input=user_input
        )

        # 토크나이즈
        inputs = tokenizer(input_prompt, return_tensors="pt").to("cuda")

        # 모델에 입력
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                do_sample=False,
                num_beams=5, 
                early_stopping=True, 
                repetition_penalty=1.1
            )

        # 토큰 디코딩 및 정리된 결과만 추가
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_output = decoded_output.split('assistant<|end_header_id>')[-1].strip()  # 불필요한 부분 제거
        # 결과만 추가
        results.append(cleaned_output)

        # 바로 출력
        if labels:
            print(f"\nSentence: {sentence}")
            print(f"Label: {labels[idx]}")
            print(f"Prediction: {cleaned_output}")
    
    return results

# 2. 추론 및 결과 저장 함수
def run_inference_and_save_results(input_csv_path, output_csv_path, model_path, system_prompts_en, language='en'):
    # 데이터 로드
    data = pd.read_csv(input_csv_path)
    
    # 문장과 레이블 분리 (영어/한국어에 따라 분리)
    if language == 'en':
        sentences = data['Sentence'].tolist()  # 추론할 문장 리스트
        labels = data['Idiom'].tolist()  # 실제 레이블 (정답)
    else:
        sentences = data['KR_Sentence'].tolist()  # 한국어 문장
        labels = data['KR_Idiom'].tolist()  # 실제 레이블 (한국어 관용구)

    # 모델 및 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # 추론 실행
    print(f"Running inference with model: {model_path} ({language})")
    model_predictions = inference(model_path, tokenizer, sentences, system_prompts_en, labels, 'en') # en으로 input language 통일 (학습때도 그랬음)

    # 결과 데이터프레임 생성 및 저장
    result_df = pd.DataFrame({
        'Sentence': sentences,
        'Label': labels,  
        'Model_Prediction': model_predictions  
    })
    result_df.to_csv(output_csv_path, index=False)
    print(f"\nResults saved to {output_csv_path}")

    return result_df

# 3. 실행
if __name__ == "__main__":
    model_path = "/data/uijih/detection/saveded_instruct-full-detection-2"
    #model_path = "meta-llama/Meta-Llama-3.1-70B-Instruct"
    input_csv_path = "/data/uijih/previous/Seed50_for_Parallel_Dataset_ENKR_idiomKB_0.8_example.csv"
    output_csv_path_en = "./8b-inst-full-en-2.csv"  
    output_csv_path_kr = "./8b-inst-full-kr-2.csv"  

    # 학습 때 사용했던 영어 시스템 프롬프트만 사용 (일단 하나만)
    system_prompts_en = [
        "Detect if there is an idiom in the following sentence. If there is, return only the detected idiom in its original form. If there are no idioms, answer 'none'."
        # "Check if the following sentence contains any idioms. If so, return the idiom only. If none, respond with 'none'.",
        # "Look for an idiom in the following sentence and return the idiom only if found. Otherwise, return 'none'."
    ]

    result_df_en = run_inference_and_save_results(input_csv_path, output_csv_path_en, model_path, system_prompts_en, language='en')    
    result_df_kr = run_inference_and_save_results(input_csv_path, output_csv_path_kr, model_path, system_prompts_en, language='kr')

    # Display the DataFrames for visualization in Jupyter
    # import ace_tools as tools
    # tools.display_dataframe_to_user(name="English Inference Results", dataframe=result_df_en)
    # tools.display_dataframe_to_user(name="Korean Inference Results", dataframe=result_df_kr)


# meaning을 너무 잘 맞춰서 이상하길래 위에껄로 한번 translation시켜보자 -> ㄴㄴ

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import random

# 1. 추론 함수 정의
def inference(model_path, tokenizer, input_data, system_prompts_en, labels=None, language='en'):
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
    model.eval()

    # 영어 프롬프트 템플릿
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

{user_input}<|eot_id|>
<|start_header_id|>assistant<|end_header_id>
"""

    results = []
    for idx, sentence in enumerate(input_data):
        # 영어 또는 한국어 프롬프트 사용
        system_prompt = random.choice(system_prompts_en)
        
        if language == 'en':
            user_input = f'Translate the sentence "{sentence}" into Korean.'
        # else:  # 한국어 문장에 대해 처리
        #     user_input = f'"{sentence}"라는 문장에서 관용구가 있니? 있으면 관용구만 알려줘. 없으면 "없음"이라고 대답해.'

        # 프롬프트 템플릿에 시스템 프롬프트와 사용자 입력 삽입
        input_prompt = prompt_template.format(
            system_prompt=system_prompt,
            user_input=user_input
        )

        # 토크나이즈
        inputs = tokenizer(input_prompt, return_tensors="pt").to("cuda")

        # 모델에 입력
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                do_sample=False,
                num_beams=5, 
                early_stopping=True, 
                repetition_penalty=1.1
            )

        # 토큰 디코딩 및 정리된 결과만 추가
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_output = decoded_output.split('assistant<|end_header_id>')[-1].strip()  # 불필요한 부분 제거
        # 결과만 추가
        results.append(cleaned_output)

        # 바로 출력
        if labels:
            print(f"\nSentence: {sentence}")
            print(f"Label: {labels[idx]}")
            print(f"Prediction: {cleaned_output}")
    
    return results

# 2. 추론 및 결과 저장 함수
def run_inference_and_save_results(input_csv_path, output_csv_path, model_path, system_prompts_en, language='en'):
    # 데이터 로드
    data = pd.read_csv(input_csv_path)
    
    # 문장과 레이블 분리 (영어/한국어에 따라 분리)
    if language == 'en':
        sentences = data['Sentence'].tolist()  # 추론할 문장 리스트
        labels = data['Idiom'].tolist()  # 실제 레이블 (정답)
    else:
        sentences = data['KR_Sentence'].tolist()  # 한국어 문장
        labels = data['KR_Idiom'].tolist()  # 실제 레이블 (한국어 관용구)

    # 모델 및 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # 추론 실행
    print(f"Running inference with model: {model_path} ({language})")
    model_predictions = inference(model_path, tokenizer, sentences, system_prompts_en, labels, 'en') # en으로 input language 통일 (학습때도 그랬음)

    # 결과 데이터프레임 생성 및 저장
    result_df = pd.DataFrame({
        'Sentence': sentences,
        'Label': labels,  
        'Model_Prediction': model_predictions  
    })
    result_df.to_csv(output_csv_path, index=False)
    print(f"\nResults saved to {output_csv_path}")

    return result_df

# 3. 실행
if __name__ == "__main__":
    model_path = "/data/uijih/detection/saveded_instruct-full-detection-2"
    #model_path = "meta-llama/Meta-Llama-3.1-70B-Instruct"
    input_csv_path = "/data/uijih/previous/Seed50_for_Parallel_Dataset_ENKR_idiomKB_0.8_example.csv"
    output_csv_path_en = "./8b-inst-full-en-2-trans.csv"  
    output_csv_path_kr = "./8b-inst-full-kr-2.csv"  

    # 학습 때 사용했던 영어 시스템 프롬프트만 사용 (일단 하나만)
    system_prompts_en = [
        "You are a professional translator proficient in Korean and English. Please translate the given English sentence into Korean accurately."
        # "Check if the following sentence contains any idioms. If so, return the idiom only. If none, respond with 'none'.",
        # "Look for an idiom in the following sentence and return the idiom only if found. Otherwise, return 'none'."
    ]

    result_df_en = run_inference_and_save_results(input_csv_path, output_csv_path_en, model_path, system_prompts_en, language='en')    
    #result_df_kr = run_inference_and_save_results(input_csv_path, output_csv_path_kr, model_path, system_prompts_en, language='kr')

    # Display the DataFrames for visualization in Jupyter
    # import ace_tools as tools
    # tools.display_dataframe_to_user(name="English Inference Results", dataframe=result_df_en)
    # tools.display_dataframe_to_user(name="Korean Inference Results", dataframe=result_df_kr)